# chapter 2 자연어 처리 개발 준비

*** 1장은 파이썬 설치와 관련된 사항이기때문에 넘어가고 2장부터 시작해보자.***  
  
자연어 처리를 위한 도구 = 라이브러리
 - 딥러닝 모델을 만드기 위한 라이브러리인 텔서플로우
 - 머신러닝 모델과 데이터를 다루기 위한 사이킷런
 - 자연어 데이터를 토크나이징 하기위한 NLTK, Spacy, Konlpy
 - 기타 Numpy, Pandas 등등
 
먼저 텐서플로우에 대해서 알아보자

## 1. Tensorflow

2015년 구글이 발표한 오픈소스 머신러닝 라이브러리  
일반인이 사용하기 쉽게 되어있고 파이썬을 주로 활용해 손쉽게 모델링 및 테스트해 볼수 있는 구조.  
특징을 보자
- 테이터 플로우 그래프를 통한 풍부한 표현력
- 아이디어 테스트에서 서비스단계까지 이용가능
- 계산 구조와 목표 함수만 정의하면 자동으로 미분계산은 처리
- 파이썬 / C++ 지원하고 SWIG를 통해 다양한 언어지원 가능
- 유연성과 확장성  

`텐서(Tensor)`는 N차원 매트릭스를 의미하며, Flow한다는것은 데이터 흐름 그래프를 사용해 수치연산을 하는 과정을 의미한다. 그래프의 `노드(Node)`는 수치연산, 변수, 상수를 나타내고, `에지(Edge)`는 노드사이를 이동하는 다차원 데이터 배열(tensor의 배열)을 나타낸다.  



## tf.keras.layers
딥러닝 모델을 만드는 것은 마치 블록을 하나씩 쌓아서 전체 구조를 만들어 가는 것과 비슷하다. 따라서 쉽게 블록을 바꾸고, 여러블록들의 조합을 쉽게 만들수 있다는 것은 텐서플로의 큰 ***장점***이다.

### tf.keras.layers.Dense
처음 소개한 모듈을 tf.keras.layers.Dense 이다. Dense란 신경망 구조의 가장 기본적인 형태를 의미한다. 즉 아래의 수식을 만족하는 기본적인 신경망 형태의 층을 만든느 함수다.

$$y=f(Wx + b)$$
x-입력벡터  
b=편향벡터  
W=가중치 행렬
f=활성화 함수  

dense layer를 그림으로 나타낸다면 
![Dense Layer](https://www.researchgate.net/profile/Mark_Sandler2/publication/319700841/figure/fig3/AS:538663165857797@1505438714241/An-illustration-of-a-dense-layer-that-has-a-4D-input-and-3D-output.png)

왼쪽은 input, 즉 x가 되는것이고 오른쪽은 output 즉 y가 되어진다. 중간 선들은 ***가중치를 곱하는 과정***을 의미하고 가중치는 W를 의미함. 이를 코드로 나타낸다면 
```python3
W= tf.Variable(tf.random_uniform([5,10],-1.0,1.0))
b= tf.Variable(tf.zeros[10])

y= tf.matmul(W,x)+b
```  
  
이같이 나타낼수있다. 텐서플로우에서는 이를 이미 정의해놓아서 아래 코드 한~두줄이면 가능함. 
```python3
1. 객체생성후 호출
dense = tf.keras.layers.Dense(..........)
output = dense(input)

2. 객체생성과 동시에 입력값 설정
dense = tf.keras.layers.Dense(..........)(input)
```

Dense 객체를 만들때 옵션을 주어 다양한 방법으로 만드는게 가능하다. 그럼 생성자를 살펴보자
```python3
def __init__(
    units,               # 출력값의 크기, integer or Long 값
    activation=None,     # 활성화 함수
    use_bias=True.       # 편향을 사용하지 여부, boolean 값
    kernel_initializer='glorot_uniform', # 가중치 초기화 함수
    bias_initializer='zeros',      # 편향 초기화 함수
    kernel_regularizer=None,       # 가중치 정규화 방법
    bias_regularizer=None,         # 편향 정규화 방법
    acitivity_regularizer=None,    # 출력 값 정규화 방법
    kernel_constraint=None,        # Optimizer에 의해 업데이트 된 이후 가중치에 적용되는 제약함수
    bias_constraint=None,          #                     ''            편향에 적용되는 제약함수
    **kwargs
    ):
```

필요에 따라 쓰자.  
예시를 들자면 
```python3
INPUT_SIZE=(20,1)

inputs=tf.keras.layers.Input(shape=INPUT_SIZE)
hidden=tf.keras.layers.Dense(10, activation=tf.nn.sigmoid)(inputs)
output=tf.keras.layers.Dense(2, activation=tf.nn.sigmoid)(hidden)
```

### tf.keras.layers.Dropout

신경망 모델을 만들때 생기는 여러 문제점 중 대표적인 문제점은 `과적합(Overfitting)`이다.  과적합을 해결하기 위헤 정규화를 사용하는데 그중에 대표적인 방법이 Dropout이다. Dropout를 쉽게 적용할수 있는데 dropout layer를 만들면 된다. 이것도 2가지 방법이 있는데
```
1.
dropout = tf.keras.layers.Dropout( ... )
output = dropout(input)

2.
output = tf.keras.layers.Dropout( ... )(inputs)
```

이것도 마찬가지로 인자를 보자

```python3
def __init__(
    rate,                 # 드롭아웃 확률을 정함. 0~1 사이값.
    noise_shape=None,     # 정수형 1D tensor를 받아 특정값만 드롭아웃 가능. 예를들어 이미지일때 특정 채널만 드롭아웃할수도 있다.
    seed-None,            # 드롭아웃의 seed값. 같은 seed일때는 동일한 드롭아웃의 결과를 만들어냄.
    **kwargs
    ):
```

dense layer와 dropout layer를 가지고 예시를 들자면 
```python3
INPUT_SIZE=(20,1)

inputs=tf.keras.layers.Input(shape=INPUT_SIZE)
dropout=tf.keras.layers.Dropout(rate=0.2)(inputs)   # 20%의 노드를 없앰.
hidden=tf.keras.layers.Dense(10, activation=tf.nn.sigmoid)(dropout)
output=tf.keras.layers.Dense(2, activation=tf.nn.sigmoid)(hidden)
```
tf.keras.layers.Dropout도 있으나 tf.keras.nn.Dropout도 있다.
둘의 차이는 
tf.keras.layers.Dropout(0.2) 는 20%의 노드를 없애는 것이고
tf.keras.nn.Dropout(0.2)는 80%의 노드를 없애는 것이다. 유의해서 사용하자.


### tf.keras.layers.Conv1D

이번절에서는 합성곱 연산중 Conv1D에 대해 알아보자. 합성곱 연산은 2가지 기준으로 구분할수 있다. 합성곱이 진행되는 방향과 합성곱 결과로 나오는 출력값이다. 표로 비교해보자

|   |합성곱의 방향|출력값|
|---|---|---|
|Conv1D|한 방향(가로)|1-D Array(vector)|
|Conv2D|두 방향(가로,세로)|2-D Array(vector)|
|Conv3D|세 방향(가로,세로,높이)|3-D Array(vector)|


실제 합성곱 출력값이 항상 동일하진 않다. 배치크기와 필터의 개수도 고려해야하기 때문에.  
그렇다면 CNN을 적용하는 과정을 이해해보자.  


![CNN 구조](https://t1.daumcdn.net/cfile/tistory/9913A0505ECBD00329)

해당 이미지를 보면 input 이미지에 필터를 통해서 feature를 뽑아 다시 2차원의 행렬을 만드는것을 볼수 있다.
Conv1D나 Conv3D를 썼다면 벡터나 텐서로 나왔을것이다.이렇게 입력값을 필터를 통해서 필터링하고 합성곱을 진행해 결과값을 얻는것이 Conv1d 레이어의 특징이다.

아래와 같이 생성 가능하다.
```
1.
conv1d = tf.keras.layers.Conv1D( ... )
output = conv1d(input)

2.
conv1d = tf.keras.layers.Conv1D( ... )(inputs)
```


이것도 마찬가지로 인자를 보자

```python3
def __init__(
    filters,                 # 총 몇개의 필터를 사용할지 -> 필터마다 출력이있으므로 출력의 차원수를 설정하는 것.
    kernel_size,             # 1차원이기 때문에 필터의 가로 길이만 필요함.
    strides=1,               # 스트라이드 값... 1이외의 수로 지정하면 dilation_rate를 1로 고정해야함.
    padding='valid',         # 패딩 값. 'valid' or 'same' 으로 지정.
    data_format='channels_last',   # 데이터의 표현 방법. channels_last or channels_first 둘중 하나.
                                   # channels_last = (batch, length, channels) // channels_first = (batch, channels, length)
    dilation_rate=1,         # 합성곱 사용시 dilation의 값. 1 이외의 수로 지정하면 strides 값을 1 이외값으로 지정하지 못함.
    activation=None,         # 활성화 함수
    use_bias=True,           # 편향 사용할지 여부.
    kernel_initializer='glorot_uniform',     # 가중치 초기화 함수
    bias_initializer='zeros', # 편향 초기화함수
    kernel_regularizer=None,  # 가중치 정규화 방법
    bias_regularizer=None,    # 편향 정규화방법
    activity_regularizer=None,# 출력값 정규화 방법
    kernel_constraint=None,   # optimizer 에 의해 업데이트된 후 가중치에 적용되는 부가적인 제약함수
    bias_constraint=None,     #                 ''               편향에 적용되는 부가적인 제약함수
    **kwargs
    ):
```


Conv1D와 dropout layer를 가지고 예시를 들자면 
```python3
INPUT_SIZE=(20,1)

inputs=tf.keras.layers.Input(shape=INPUT_SIZE)
dropout=tf.keras.layers.Dropout(rate=0.2)(inputs)   # 20%의 노드를 없앰.
conv=tf.keras.layers.Conv1D(
        filters=10,
        kernel_size=3,
        padding='same',
        activation=tf.nn.relu)(dropout)
```
conv2d와 conv3d의 경우도 몇가지 옵션을 제외하면 대부분 비슷하기 때문에 공식문서를 보고 비교해서 잘 알아두자...


### tf.keras.layers.MaxPool1D

CNN에서 함께 쓰이는 기법중 하나가 풀링이다. 보통 feature map 크기를 줄이거나 주요한 특징을 뽑아내기 위해 합성곱 이후에 적용하는 기법이다. 풀링에는 2가지가 쓰이는데 MaxPooling과 AvreagePooling이다. Max pooling은 feature map에서 최대값만 뽑아내는 방법이고 Average pooling은 전체값들의 평균을 뽑아내는 방식이다. 주로 `Max pooling`을 이용하므로 알아두자  

MaxPooling도 3가지 형태로 구분되어있다. MaxPool1D,MaxPool2D,MaxPool3D가 있다.  
자연어 처리에선 MaxPool1D가 주로 사용되므로 MaxPool1D를 보도록 하자.

사용방법은 앞 클래스들과 동일하다  
```
1.
maxpool1d = tf.keras.layers.MaxPool1D( ... )
output = maxpool1d(input)

2.
maxpool1d = tf.keras.layers.MaxPool1D( ... )(inputs)
```


이것도 마찬가지로 인자를 보자

```python3
def __init__(
    pool_size=2,            # 풀링을 적용할 필터의 크기를 뜻한다. 정수값
    strides=None,           # 적용한 스트라이드의 값.
    padding='valid',        # 패딩방법을 정한다 'valid' or 'same'
    data_format=None        # channels_first or channels_last. Conv1D 와 같음.
    **kwargs
    ):
```


위에서 썻던 방식들과 적용해서 예시를 보자. 입력값이 합성섭과 맥스 풀링을 사용한 후 완전 연결 계층을 통해 최종출력값이 나오는 구조를 만들어 보자. 드롭아웃을 적용하고 맥스풀링 결과값을 완전 연결 계층으로 연결하기위해 *행렬이었던것을 벡터로 만들어야 한다.* 이때는 `tf.keras.layers.Flatten`을 사용한다. flatten은 별다른 인자값설정 없이도 사용할수 있으므로 다음 코드를 보자.  

```python3
INPUT_SIZE=(20,1)

inputs=tf.keras.layers.Input(shape=INPUT_SIZE)
dropout=tf.keras.layers.Dropout(rate=0.2)(inputs)   # 20%의 노드를 없앰.
conv=tf.keras.layers.Conv1D(
        filters=10,
        kernel_size=3,
        padding='same',
        activation=tf.nn.relu)(dropout)
max_pool=tf.keras.layers.MaxPool1D(pool_size=3, padding='same')(conv)
flatten=tf.keras.layers.Flatten()(max_pool)
hidden=tf.keras.layers.Dense(50, activation=tf.nn.sigmoid)(flatten)
output=tf.keras.layers.Dense(10, activation=tf.nn.sigmoid)(hidden)
```



# 2. Tensorflow 2.0  

텐서플로우 2.0으로 버전업 되면서 업데이트 된 부분을 요약하면 다음과 같다.  

- API 정리 (api clean up)
- 이거모드 (eager execution)
- 전역 매터니즘 제거 (No more globals)
- 세션을 대신하는 함수 (functions, not sessions)

### API 정리  

1 버전에서는 같은 기능을 수행하는 다양한 API가 다양한 패키지에 혼재해 있었다면 2버전에서는 다양한 API를 명료하고 사용하기 편하도록 동일한 기능의 다양한 API로 통합하고 사용하지 않는 API는 제거했다.

### 이거 모드(eager execution)   
기존에는 텐서플로우 API를 이용해 그래프를 만든 후에 별도의 세션을 통해 해당 그래프를 실행하는 방법이었다. 하지만 버전업 되면서 파이썬과 동일한 이거모드로 실행되기 때문에 연산을 구성하면서 바로 값을 확인할수 있게 되었다.

## 모델 구축
텐서플로우는 케라스를 이용해 모델을 구축하는것을 권장한다. 고수준(high-level) API로써 간편하고 유연하고 높은 성능을 보여준다.  
케라스로 모델을 구축하는방법은 크게 3가지이다.  
- Sequential API  
- Functional API  
- Function/ Sequential API  
  - \+ Custom API  
- Subclassing(Custom Model)  

이책은 Subclassing을 주로 이용해 모델을 구축할것이다. 일단 하나하나 알아보자.
